In [1]:
# import sys
# !conda install --yes --prefix {sys.prefix} pingouin

In [2]:
import datetime
import numpy as np
import math
import os
import pandas as pd
import pingouin as pg
import re
import seaborn as sns
import json
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    

os.chdir(r'C:\Users\User\GitHub\WebET_Analysis')
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [3]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Raw Data

## Search for specific subjects

In [4]:
#path = 'data_cognition'
#subject_files = os.listdir(path)
#for i in range(0, len(subject_files)):
#    thisSubject = open(path + "/" + subject_files[i]).read()
#    if thisSubject.find('5c0e4e3d87e876000151cfec') > (-1):
#        print(subject_files[i])

## Read CSV from String

In [5]:
def cleanhtml(raw_html):
    # https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
    cleanr = re.compile('<.*?>')
    cleanText = re.sub(cleanr, '', raw_html)
    return cleanText

def cleanETText(text):
    textWithinBrackets = re.findall(re.compile('\[.*?\]'), text)
    output = text
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = re.sub(",", "$", old)
        output = output.replace(old, new)
    return output

def cleanSurveyText(text):
    output = text
    textWithinBrackets = re.findall(re.compile('\{.*?\}'), text)
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = old.replace(',', '§')
        output = output.replace(old, new)
    return output

def compileData(path):
    subject_files = os.listdir(path)
    all_subjects = []
    for i in range(0, len(subject_files)):
        csv_thisSubject = open(path + "/" + subject_files[i]).read()
        csv_thisSubject = cleanhtml(csv_thisSubject)
        csv_thisSubject = cleanETText(csv_thisSubject)
        csv_thisSubject = cleanSurveyText(csv_thisSubject)
        all_subjects.append(pd.read_csv(StringIO(csv_thisSubject)))
    output = pd.concat(all_subjects).reset_index(drop=True)
    return output

#data_yang = compileData("data_yang2020WG")
# Custom modifications
#data_yang.loc[0:515, 'run_id'] = 0
#data_yang = data_yang.loc[~data_yang['run_id'].isin([4, 18]), :]
#data_yang['run_id'] = data_yang['run_id'] * 1000
#data_yang = data_yang.rename(columns={'eyeData': 'et_data'})
#data_cognition = compileData("data_cognition")

#data_raw = data_yang.append(data_cognition)
#data_raw['run_id'].unique()

data_raw = compileData("data_cognition")
# Custom modifications
data_raw['run_id'].unique()

array([1, 103, 105, 106, 108, 11, 117, 12, 121, 124, 125, 126, 128, 13,
       130, 131, 14, 19, 24, 25, 28, 3, 30, 32, 36, 37, 38, 4, 41, 42, 43,
       45, 47, 48, 49, 5, 50, 51, 54, 56, 57, 58, 59, 6, 61, 63, 66, 67,
       7, 70, 72, 74, 75, 80, 81, 83, 85, 88, 9, 90, 91, 92, 93, 94, 96,
       97, 98, 99], dtype=object)

### Exclude empty studies

In [6]:
data_raw = data_raw.loc[data_raw['trial_index']>0, :]

### survey data

In [7]:
def cleanOptionalNote(text):
    optionalNoteText = re.findall(re.compile('optionalNote":.*?\}'), text)
    if len(optionalNoteText) < 1:
        output = text
    else:
        old = optionalNoteText[0]
        new = old.replace('§', ' ')
        output = text.replace(old, new)
    return output


def surveyStringToFrame(subject, string):
    string = cleanOptionalNote(string)
    string = re.sub("""{""", '', string)
    string = re.sub("""}""", '', string)
    string = re.sub('"', '', string)
    string = re.sub('§', '$', string)
    output = pd.read_csv(StringIO(string),
                         sep=":",
                         lineterminator="$",
                         header=None,
                         index_col=0
                         ).transpose()
    return output


def surveyData_thisSubject(data):
    df_thisSubject = data.loc[
        (pd.notna(data["responses"])) &
        (data["responses"] != '"'), :].reset_index()
    subject = df_thisSubject['run_id'].unique()[0]
    output = []
    for i in range(0, len(df_thisSubject)):
        output.append(
            surveyStringToFrame(subject,
                                df_thisSubject.loc[i, 'responses'])
        )

    output = pd.concat(output, axis=1)
    output['run_id'] = subject
    return output

def addSurveyData(data):
    surveyData_allSubjects = pd.DataFrame(columns=[
            'prolificID', 'age', 'gender', 'ethnic', 'sight', 
            'glasses', 'degree', 'eyeshadow', 'masquara', 'eyeliner', 
            'browliner', 'vertPosition', 'triedChin', 'keptHead', 
            'optionalNote', 'run_id'])

    for subject in data['run_id'].unique():
        surveyData_allSubjects = \
            surveyData_allSubjects.append(
                surveyData_thisSubject(
                    data.loc[data['run_id']==subject, ['run_id', 'responses']]
                )
            )
    # print(surveyData_allSubjects)
    
    data = data.merge(surveyData_allSubjects, on='run_id', how='left')
    data = data.drop(columns='responses')
    return(data)
    
data_raw = addSurveyData(data_raw)
print(data_raw['run_id'].unique())
print(data_raw.columns)

[1 103 106 108 11 12 124 125 126 128 13 130 131 14 19 24 25 28 30 32 36 37
 38 4 41 42 43 45 47 48 49 5 54 56 58 59 6 61 63 66 67 7 70 72 74 75 80 81
 83 85 88 9 90 91 92 93 94 96 97 98 99]
Index(['run_id', 'condition', 'rt', 'stimulus', 'key_press', 'trial_type',
       'trial_index', 'time_elapsed', 'internal_node_id', 'subject',
       'chinFirst', 'choiceTask_amountLeftFirst', 'webcam_label', 'webcam_fps',
       'webcam_height', 'webcam_width', 'button_pressed', 'window_width',
       'window_height', 'chin', 'success', 'x_pos', 'y_pos', 'task_nr',
       'et_data', 'trial_duration', 'option_topLeft', 'option_bottomLeft',
       'option_topRight', 'option_bottomRight', 'recorded_at', 'ip',
       'user_agent', 'device', 'browser', 'browser_version', 'platform',
       'platform_version', 'Unnamed: 2', 'chosenAmount', 'chosenDelay',
       'prolificID', 'age', 'gender', 'ethnic', 'sight', 'glasses', 'degree',
       'eyeshadow', 'masquara', 'eyeliner', 'browliner', 'vertPosition',


In [8]:
def convertToNumeric(data, columns):
    data[columns] = data[columns].apply(pd.to_numeric, errors='coerce')
    return data


data_raw = convertToNumeric(data_raw, ['age'])
data_raw['degree'] = data_raw['degree'].replace(
        [
            'College / Undergraduate / Bachelor',
            'High School',
            'Graduate / PhD / Master',
            'Middle School'
        ],
    [
            'college',
            'highSchool',
            'grad',
            'middle'
        ]
)

columns = [
            'chinFirst',
            'eyeshadow', 
            'masquara',
            'eyeliner',
            'browliner',
            'triedChin', 
            'keptHead',
        ]

data_raw[columns] = data_raw[columns].replace({'no': 0, 'yes': 1}) 

data_raw['run_id'].unique()

array([1, 103, 106, 108, 11, 12, 124, 125, 126, 128, 13, 130, 131, 14, 19,
       24, 25, 28, 30, 32, 36, 37, 38, 4, 41, 42, 43, 45, 47, 48, 49, 5,
       54, 56, 58, 59, 6, 61, 63, 66, 67, 7, 70, 72, 74, 75, 80, 81, 83,
       85, 88, 9, 90, 91, 92, 93, 94, 96, 97, 98, 99], dtype=object)

# data_trial

In [9]:
data_trial = data_raw.loc[
    :,
    [
        'run_id', 'subject', 'prolificID', 'chinFirst', 
        'trial_index',
        'trial_type', 'task_nr', 
        'rt', 'stimulus', 'key_press', 
        'time_elapsed', 'trial_duration', 
        'window_width', 'window_height', 'success', 

        'chin', 'x_pos', 'y_pos', 
        
        'choiceTask_amountLeftFirst', 
        'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight', 
        'chosenAmount', 'chosenDelay',
    ]
]

## Clean

### Failed Webgazer Setup

In [10]:
subjects_failedSetups = []
for subject in data_trial['run_id'].unique():
    maxTrialIndex = data_trial.loc[data_raw['run_id']==subject, ['trial_index']].max()
    if int(maxTrialIndex) < 15:
        subjects_failedSetups.append(subject)

data_trial.loc[data_trial['run_id'].isin(subjects_failedSetups), 
    [
        'run_id', 'prolificID'
    ]
].drop_duplicates()    

,run_id,prolificID
1036,106,5d485e8415055400194b707f
1038,108,55b237e6fdf99b19ea79d2f7
2079,124,56b7a271e77ebe000bbeff49
4684,14,5ec2cdfd1a17930ddf6f1443
12984,49,5ecfc227f036c902457fc44c
13507,54,5f94d146f65f56336a61cc16
18338,72,600324757b530e39903effc0
20414,83,600324757b530e39903effc0
20935,88,5ecfc227f036c902457fc44c
25608,99,55b237e6fdf99b19ea79d2f


In [11]:
data_trial = data_trial.loc[~data_trial['run_id'].isin(subjects_failedSetups), :]
print(data_trial['run_id'].unique())

[1 103 11 12 125 126 128 13 130 131 19 24 25 28 30 32 36 37 38 4 41 42 43
 45 47 48 5 56 58 59 6 61 63 66 67 7 70 74 75 80 81 85 9 90 91 92 93 94 96
 97 98]


### Duplicate Prolific ID's

In [12]:
data_trial.loc[data_trial['run_id']==2, 'prolificID'] = 'Tim2'


duplicates = data_trial.loc[:, ['prolificID', 'trial_index']].duplicated()
duplicateSubjects = data_trial.loc[duplicates, 'run_id'].unique()

if len(duplicateSubjects) > 0:
    print('! Attention: Duplicate subjects: Check out the following: \n')
    print(duplicateSubjects)
else:
    print('Success: No duplicate subjects found')

Success: No duplicate subjects found


### Choice data

In [13]:
# Cleaning
data_trial.loc[: , 
       [
           'option_topLeft',
           'option_bottomLeft', 
           'option_topRight', 
           'option_bottomRight', 
           'chosenAmount', 
           'chosenDelay'
       ] 
    ] = data_trial.loc[:, 
       [
           'option_topLeft',
           'option_bottomLeft', 
           'option_topRight', 
           'option_bottomRight', 
           'chosenAmount', 
           'chosenDelay'
       ] 
    ].replace(['Today', 'Tomorrow', '7 days', '15 days', '30 days', '90 days', '180 days'], 
             [0, 1, 7, 15, 30, 90, 180]) \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) 

### Numerics

In [14]:
data_trial = convertToNumeric(data_trial, 
    [
        'run_id', 'subject', 'chinFirst', 'chin', 'task_nr', 'trial_index', # Int
        'key_press', 
        'x_pos', 'y_pos', 'time_elapsed', 'trial_duration',
        'rt',
        'window_width', 'window_height', 
        'option_topLeft',
        'option_bottomLeft', 
        'option_topRight', 
        'option_bottomRight', 
        'chosenAmount', 
        'chosenDelay'
    ]
)

## Add variables

### Window Size

In [15]:
def addWindowSize(data):
    output = data
    if "window_width_max" in data.columns:
        print('window width_max already added')
    else:
        grouped = data.groupby(["run_id", "subject"])["window_width", "window_height"].max().reset_index()
        grouped.columns = ["run_id", "subject", "window_width_max", "window_height_max"]
        grouped['window_diagonal_max'] = np.sqrt(grouped['window_width_max']**2 + grouped['window_height_max']**2)
        output = data.merge(grouped, 
                            on=['run_id', "subject"],
                            how='left')
    return output

data_trial = addWindowSize(data_trial)
data_trial['window_diagonal'] =  \
    np.sqrt(data_trial['window_width']**2 + 
    data_trial['window_height']**2)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


### Trial Duration

In [16]:
def checkTimeDeviation(data, column1, column2, maxTimeDiffAllowed):
    diff = data[column1] - data['trial_duration_exact']
    longtrials_runID = data.loc[diff[diff > maxTimeDiffAllowed].index, 'run_id']
    longtrials_previousrunID = pd.DataFrame(data.loc[diff[diff > maxTimeDiffAllowed].index-1, 'run_id']) \
        .rename(columns={'run_id':'previous_run_id'})
    longtrials_previousrunID.index = longtrials_runID.index
    compare_runIDs = pd.concat([longtrials_runID, longtrials_previousrunID], axis=1)

    if sum(compare_runIDs['run_id'] == compare_runIDs['previous_run_id']) > 0: 
        print(column1 + ' and ' + column2 + ' show a deviation of ' +
              '>' + str(maxTimeDiffAllowed) + 
              ' ms. Please check on the following indices: \n')
        print(compare_runIDs.loc[(compare_runIDs['run_id'] == compare_runIDs['previous_run_id']), :].index)

    else:
        print('Success! ' + column1 + ' and ' + column2 + ' do not deviate by ' +
              '>' + str(maxTimeDiffAllowed) + 'ms.')
        
def exactTrialDuration(data):
    output = data
    output["t_startTrial"] = pd.concat([pd.Series([0]), output["time_elapsed"]], ignore_index=True)
    output["trial_duration_exact"] = output.loc[:, ("time_elapsed")] - output.loc[:, ("t_startTrial")]
    output.drop(len(output)-1)
    
    checkTimeDeviation(data, 'rt', 'trial_duration_exact', 50)
    checkTimeDeviation(data, 'trial_duration', 'trial_duration_exact', 50)
    
    return output

data_trial = exactTrialDuration(data_trial)

Success! rt and trial_duration_exact do not deviate by >50ms.
Success! trial_duration and trial_duration_exact do not deviate by >50ms.


### withinTaskIndex

In [17]:
def withinTaskIndex(data): 
    allTrialIndices = []
    for subject in data["run_id"].unique():
        df_subj = data.loc[data['run_id']==subject, :]
        
        for trial_type in df_subj['trial_type'].unique():
            df_trial = df_subj.loc[df_subj['trial_type']==trial_type, :]
                
            for task_nr in df_trial["task_nr"].unique():
                df_thisTask = df_trial.loc[df_trial['task_nr']==task_nr, 
                         [
                             'run_id',
                             'trial_index'
                         ]
                    ] \
                    .drop_duplicates() \
                    .reset_index(drop=True)
        
                df_thisTask['withinTaskIndex'] = df_thisTask.index + 1
                allTrialIndices.append(df_thisTask)
    allTrialIndices = pd.concat(allTrialIndices).reset_index(drop=True)
    return allTrialIndices

def addWithinTaskIndex(data):
    if 'withinTaskIndex' in data.columns: 
        print('withinTaskIndex already added')
    else:
        newIndices = withinTaskIndex(data) \
            .reset_index(drop=True)
        data = data.merge(newIndices, 
                          on = ['run_id', 'trial_index'], 
                          how = 'left')
    return data

data_trial = addWithinTaskIndex(data_trial)

In [18]:
data_trial.columns

Index(['run_id', 'subject', 'prolificID', 'chinFirst', 'trial_index',
       'trial_type', 'task_nr', 'rt', 'stimulus', 'key_press', 'time_elapsed',
       'trial_duration', 'window_width', 'window_height', 'success', 'chin',
       'x_pos', 'y_pos', 'choiceTask_amountLeftFirst', 'option_topLeft',
       'option_bottomLeft', 'option_topRight', 'option_bottomRight',
       'chosenAmount', 'chosenDelay', 'window_width_max', 'window_height_max',
       'window_diagonal_max', 'window_diagonal', 't_startTrial',
       'trial_duration_exact', 'withinTaskIndex'],
      dtype='object')

# data_trial_fixCal

In [19]:
data_trial_fixCal = data_trial.loc[
    data_trial['trial_type'].isin(['eyetracking-calibration', 'eyetracking-fix-object']),
    [
        'run_id', 'chinFirst', 
        'trial_type', 'trial_index', 'withinTaskIndex', 
        'trial_duration', 'trial_duration_exact',
        'chin', 'x_pos', 'y_pos', 'task_nr',
        'window_height', 'window_width']
].reset_index(drop=True)

## Add variables

### positionIndex

In [20]:
def multiply(x):
    return x*100

def positionIndex(data):
    allPositionIndices = []
    for subject in data["run_id"].unique():
        df_subj = data.loc[
                (data['run_id']==subject) &
                (data['trial_type'].isin(['eyetracking-calibration', 'eyetracking-fix-object'])), 
            :]
        
        for trial_type in df_subj['trial_type'].unique():
            df_trial = df_subj.loc[df_subj['trial_type']==trial_type, :]
                
            for task_nr in df_trial["task_nr"].unique():
                df_thisTask = df_trial.loc[
                        df_trial['task_nr']==task_nr, 
                        ['run_id', 'trial_index', 'x_pos', 'y_pos']
                    ] \
                    .drop_duplicates() \
                    .reset_index(drop=True)

                df_thisTask['positionIndex'] = df_thisTask.loc[:, ['x_pos', 'y_pos']] \
                    .apply(multiply) \
                    .astype(int) \
                    .astype(str) \
                    .apply(''.join, 1)
                df_thisTask['positionIndex'] = df_thisTask['positionIndex'].astype(int)
                df_thisTask['positionIndex'] = df_thisTask.loc[:, 'positionIndex'] \
                    .replace(
                            np.sort(df_thisTask['positionIndex'].unique()), 
                            range(0, len(df_thisTask['positionIndex'].unique()))
                        )        
                allPositionIndices.append(df_thisTask)
                
    allPositionIndices = pd.concat(allPositionIndices).reset_index(drop=True)
    return allPositionIndices

def addPositionIndex(data):
    if 'positionIndex' in data.columns: 
        print('positionIndex already added')
    else:
        newIndices = positionIndex(data) \
            .reset_index(drop=True)
        data = data.merge(newIndices, 
                          on = ['run_id', 'trial_index', 'x_pos', 'y_pos'], 
                          how = 'left')
    return data

data_trial_fixCal = addPositionIndex(data_trial_fixCal)

# data_trial_choice

In [21]:
data_trial_choice = data_trial.loc[
    data_trial['trial_type']=='eyetracking-choice', 
    [
        'run_id', 'chinFirst', 
        'task_nr', 
        'trial_index', 'trial_type', 'withinTaskIndex', 
        'choiceTask_amountLeftFirst', 
        'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight',
        'key_press', 'trial_duration_exact',
        'window_width', 'window_height'
    ]
]

## Choice variables (aLL, aSS, tLL, etc.)

In [22]:
def addChoiceVariables(data):
    data['amountLeft']=0
    data.loc[(data['choiceTask_amountLeftFirst']==1) &
             (data['withinTaskIndex'] <41), 
             'amountLeft'] = 1

    data['aSS'] = \
        data.loc[:, ["option_topRight", "option_bottomRight"]].values.min(1)
    data.loc[data['amountLeft'] == 1, 'aSS'] = \
        data.loc[data['amountLeft'] == 1, ["option_topLeft", "option_bottomLeft"]].values.min(1)
    
    data['aLL'] = \
        data.loc[:, ["option_topLeft", "option_bottomLeft"]].values.max(1)
    data.loc[data['amountLeft'] == 0, 'aLL'] = \
        data.loc[data['amountLeft'] == 0, ["option_topRight", "option_bottomRight"]].values.max(1)

    data.loc[:, "tSS"] = 0 
    
    data['tLL'] = \
        data.loc[:, ["option_topRight", "option_bottomRight"]].values.max(1)
    data.loc[data['amountLeft'] == 0, 'tLL'] = \
        data.loc[data['amountLeft'] == 0, ["option_topLeft", "option_bottomLeft"]].values.max(1)
    
    data['LL_top'] = (data["option_topLeft"] > data["option_bottomLeft"]).astype(int)
        
    data["choseTop"] = 0
    data.loc[(data["key_press"]==38), "choseTop"] = 1

    data["choseLL"] = 0
    data.loc[(data["choseTop"]==1) & (data["LL_top"] == 1), "choseLL"] = 1
    
    return(data)

data_trial_choice = addChoiceVariables(data_trial_choice)

## k

In [23]:
def k(aLL, aSS, tLL):
    k = ((aLL / aSS) - 1) / tLL
    return k

data_trial_choice['k'] = k(data_trial_choice['aLL'], data_trial_choice['aSS'], data_trial_choice['tLL']) 
data_trial_choice

,run_id,chinFirst,task_nr,trial_index,trial_type,withinTaskIndex,choiceTask_amountLeftFirst,option_topLeft,option_bottomLeft,option_topRight,...,window_height,amountLeft,aSS,aLL,tSS,tLL,LL_top,choseTop,choseLL,k
144,1,1.0,2.0,145.0,eyetracking-choice,1.0,0.0,30.0,0.0,4.0,...,1080.0,0,3.0,4.0,0,30.0,1,1,1,0.011111
147,1,1.0,2.0,148.0,eyetracking-choice,2.0,0.0,30.0,0.0,5.0,...,1080.0,0,3.5,5.0,0,30.0,1,1,1,0.014286
150,1,1.0,2.0,151.0,eyetracking-choice,3.0,0.0,0.0,180.0,2.5,...,1080.0,0,2.5,5.0,0,180.0,0,0,0,0.005556
153,1,1.0,2.0,154.0,eyetracking-choice,4.0,0.0,180.0,0.0,4.5,...,1080.0,0,4.0,4.5,0,180.0,1,0,0,0.000694
156,1,1.0,2.0,157.0,eyetracking-choice,5.0,0.0,15.0,0.0,4.5,...,1080.0,0,4.0,4.5,0,15.0,1,0,0,0.008333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25552,98,0.0,3.0,496.0,eyetracking-choice,76.0,0.0,0.5,4.5,0.0,...,1080.0,0,0.0,30.0,0,4.5,0,0,0,inf
25555,98,0.0,3.0,499.0,eyetracking-choice,77.0,0.0,2.5,5.0,0.0,...,1080.0,0,0.0,30.0,0,5.0,0,0,0,inf
25558,98,0.0,3.0,502.0,eyetracking-choice,78.0,0.0,2.0,5.0,0.0,...,1080.0,0,0.0,7.0,0,5.0,0,0,0,inf
25561,98,0.0,3.0,505.0,eyetracking-choice,79.0,0.0,4.5,0.5,90.0,...,1080.0,0,0.0,90.0,0,4.5,1,1,1,inf


# data_et

## Compile

In [24]:
def reformatYang(text):
    text = text.replace('$', ',')
    text = text.replace("relative-x", "x")
    text = text.replace("relative-y", 'y')
    text = text.replace('elapse-time', 't')
    text = (text[11:len(text)-1])
    return(text)

for subject in [0, 11000, 21000, 3000, 8000]:
    for i in data_raw.loc[(data_raw['run_id']==subject) &
                      (data_raw['et_data']!='"'), :].index:
        data_raw.loc[i, 'et_data'] = reformatYang(data_raw.loc[i, 'et_data'])

In [ ]:
def textToDataframe(text):
    text = text.replace('$', ',')
    dataframe = pd.read_json(text, orient='records')
    return(dataframe)

def extractEyetrackingData(data):
    data_eyetracking = pd.DataFrame(columns=['x', 'y', 't'])
    data["et_data"] = data['et_data'].apply(str)

    for i in data.loc[~(data['et_data'].isin(['"', '[]', 'nan'])), :].index:
        print('Start decoding Index ' + str(i))
        df = textToDataframe(data.loc[i, 'et_data'])        

        df["t_task"] = (df.loc[:, "t"] - df.loc[0, "t"])
        df['run_id'] = data.loc[i, 'run_id']
        df['trial_index'] = data.loc[i, 'trial_index']

        data_eyetracking = data_eyetracking.append(pd.DataFrame(data=df), ignore_index=True)
        print('Index ' + str(i) + ' extracted')
    return(data_eyetracking)

data_et = extractEyetrackingData(data_raw.loc[data_raw['run_id'].isin(data_trial['run_id'].unique()), :])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Start decoding Index 17
Index 17 extracted
Start decoding Index 19
Index 19 extracted
Start decoding Index 21
Index 21 extracted
Start decoding Index 23
Index 23 extracted
Start decoding Index 25
Index 25 extracted
Start decoding Index 27
Index 27 extracted
Start decoding Index 29
Index 29 extracted
Start decoding Index 31
Index 31 extracted
Start decoding Index 33
Index 33 extracted
Start decoding Index 35
Index 35 extracted
Start decoding Index 37
Index 37 extracted
Start decoding Index 39
Index 39 extracted
Start decoding Index 41
Index 41 extracted
Start decoding Index 43
Index 43 extracted
Start decoding Index 45
Index 45 extracted
Start decoding Index 47
Index 47 extracted
Start decoding Index 49
Index 49 extracted
Start decoding Index 51
Index 51 extracted
Start decoding Index 53
Index 53 extracted
Start decoding Index 55
Index 55 extracted
Start decoding Index 57
Index 57 extracted
Start decoding Index 59
Index 59 extracted
Start decoding Index 61
Index 61 extracted
Start decod

Index 348 extracted
Start decoding Index 349
Index 349 extracted
Start decoding Index 351
Index 351 extracted
Start decoding Index 352
Index 352 extracted
Start decoding Index 354
Index 354 extracted
Start decoding Index 355
Index 355 extracted
Start decoding Index 357
Index 357 extracted
Start decoding Index 358
Index 358 extracted
Start decoding Index 360
Index 360 extracted
Start decoding Index 361
Index 361 extracted
Start decoding Index 363
Index 363 extracted
Start decoding Index 364
Index 364 extracted
Start decoding Index 366
Index 366 extracted
Start decoding Index 367
Index 367 extracted
Start decoding Index 369
Index 369 extracted
Start decoding Index 370
Index 370 extracted
Start decoding Index 372
Index 372 extracted
Start decoding Index 373
Index 373 extracted
Start decoding Index 375
Index 375 extracted
Start decoding Index 376
Index 376 extracted
Start decoding Index 378
Index 378 extracted
Start decoding Index 379
Index 379 extracted
Start decoding Index 381
Index 381 

Index 765 extracted
Start decoding Index 766
Index 766 extracted
Start decoding Index 768
Index 768 extracted
Start decoding Index 769
Index 769 extracted
Start decoding Index 771
Index 771 extracted
Start decoding Index 772
Index 772 extracted
Start decoding Index 786
Index 786 extracted
Start decoding Index 787
Index 787 extracted
Start decoding Index 789
Index 789 extracted
Start decoding Index 790
Index 790 extracted
Start decoding Index 792
Index 792 extracted
Start decoding Index 793
Index 793 extracted
Start decoding Index 795
Index 795 extracted
Start decoding Index 796
Index 796 extracted
Start decoding Index 798
Index 798 extracted
Start decoding Index 799
Index 799 extracted
Start decoding Index 801
Index 801 extracted
Start decoding Index 802
Index 802 extracted
Start decoding Index 804
Index 804 extracted
Start decoding Index 805
Index 805 extracted
Start decoding Index 807
Index 807 extracted
Start decoding Index 808
Index 808 extracted
Start decoding Index 810
Index 810 

Index 1119 extracted
Start decoding Index 1121
Index 1121 extracted
Start decoding Index 1123
Index 1123 extracted
Start decoding Index 1125
Index 1125 extracted
Start decoding Index 1127
Index 1127 extracted
Start decoding Index 1129
Index 1129 extracted
Start decoding Index 1131
Index 1131 extracted
Start decoding Index 1133
Index 1133 extracted
Start decoding Index 1135
Index 1135 extracted
Start decoding Index 1146
Index 1146 extracted
Start decoding Index 1147
Index 1147 extracted
Start decoding Index 1149
Index 1149 extracted
Start decoding Index 1150
Index 1150 extracted
Start decoding Index 1152
Index 1152 extracted
Start decoding Index 1153
Index 1153 extracted
Start decoding Index 1155
Index 1155 extracted
Start decoding Index 1156
Index 1156 extracted
Start decoding Index 1158
Index 1158 extracted
Start decoding Index 1159
Index 1159 extracted
Start decoding Index 1161
Index 1161 extracted
Start decoding Index 1162
Index 1162 extracted
Start decoding Index 1164
Index 1164 ex

...

Index 11773 extracted
Start decoding Index 11774
Index 11774 extracted
Start decoding Index 11776
Index 11776 extracted
Start decoding Index 11777
Index 11777 extracted
Start decoding Index 11779
Index 11779 extracted
Start decoding Index 11780
Index 11780 extracted
Start decoding Index 11782
Index 11782 extracted
Start decoding Index 11783
Index 11783 extracted
Start decoding Index 11785
Index 11785 extracted
Start decoding Index 11786
Index 11786 extracted
Start decoding Index 11788
Index 11788 extracted
Start decoding Index 11789
Index 11789 extracted
Start decoding Index 11791
Index 11791 extracted
Start decoding Index 11792
Index 11792 extracted
Start decoding Index 11794
Index 11794 extracted
Start decoding Index 11795
Index 11795 extracted
Start decoding Index 11797
Index 11797 extracted
Start decoding Index 11798
Index 11798 extracted
Start decoding Index 11800
Index 11800 extracted
Start decoding Index 11801
Index 11801 extracted
Start decoding Index 11803
Index 11803 extracte

Index 12092 extracted
Start decoding Index 12094
Index 12094 extracted
Start decoding Index 12095
Index 12095 extracted
Start decoding Index 12097
Index 12097 extracted
Start decoding Index 12098
Index 12098 extracted
Start decoding Index 12100
Index 12100 extracted
Start decoding Index 12101
Index 12101 extracted
Start decoding Index 12103
Index 12103 extracted
Start decoding Index 12104
Index 12104 extracted
Start decoding Index 12106
Index 12106 extracted
Start decoding Index 12107
Index 12107 extracted
Start decoding Index 12109
Index 12109 extracted
Start decoding Index 12110
Index 12110 extracted
Start decoding Index 12112
Index 12112 extracted
Start decoding Index 12113
Index 12113 extracted
Start decoding Index 12115
Index 12115 extracted
Start decoding Index 12116
Index 12116 extracted
Start decoding Index 12118
Index 12118 extracted
Start decoding Index 12119
Index 12119 extracted
Start decoding Index 12121
Index 12121 extracted
Start decoding Index 12122
Index 12122 extracte

Index 12375 extracted
Start decoding Index 12377
Index 12377 extracted
Start decoding Index 12379
Index 12379 extracted
Start decoding Index 12381
Index 12381 extracted
Start decoding Index 12383
Index 12383 extracted
Start decoding Index 12385
Index 12385 extracted
Start decoding Index 12387
Index 12387 extracted
Start decoding Index 12389
Index 12389 extracted
Start decoding Index 12391
Index 12391 extracted
Start decoding Index 12393
Index 12393 extracted
Start decoding Index 12395
Index 12395 extracted
Start decoding Index 12397
Index 12397 extracted
Start decoding Index 12399
Index 12399 extracted
Start decoding Index 12401
Index 12401 extracted
Start decoding Index 12403
Index 12403 extracted
Start decoding Index 12405
Index 12405 extracted
Start decoding Index 12407
Index 12407 extracted
Start decoding Index 12409
Index 12409 extracted
Start decoding Index 12411
Index 12411 extracted
Start decoding Index 12413
Index 12413 extracted
Start decoding Index 12415
Index 12415 extracte

Index 12757 extracted
Start decoding Index 12759
Index 12759 extracted
Start decoding Index 12760
Index 12760 extracted
Start decoding Index 12762
Index 12762 extracted
Start decoding Index 12763
Index 12763 extracted
Start decoding Index 12765
Index 12765 extracted
Start decoding Index 12766
Index 12766 extracted
Start decoding Index 12768
Index 12768 extracted
Start decoding Index 12769
Index 12769 extracted
Start decoding Index 12771
Index 12771 extracted
Start decoding Index 12772
Index 12772 extracted
Start decoding Index 12774
Index 12774 extracted
Start decoding Index 12775
Index 12775 extracted
Start decoding Index 12777
Index 12777 extracted
Start decoding Index 12778
Index 12778 extracted
Start decoding Index 12780
Index 12780 extracted
Start decoding Index 12781
Index 12781 extracted
Start decoding Index 12783
Index 12783 extracted
Start decoding Index 12784
Index 12784 extracted
Start decoding Index 12786
Index 12786 extracted
Start decoding Index 12787
Index 12787 extracte

Index 13067 extracted
Start decoding Index 13069
Index 13069 extracted
Start decoding Index 13071
Index 13071 extracted
Start decoding Index 13073
Index 13073 extracted
Start decoding Index 13075
Index 13075 extracted
Start decoding Index 13077
Index 13077 extracted
Start decoding Index 13079
Index 13079 extracted
Start decoding Index 13081
Index 13081 extracted
Start decoding Index 13092
Index 13092 extracted
Start decoding Index 13093
Index 13093 extracted
Start decoding Index 13095
Index 13095 extracted
Start decoding Index 13096
Index 13096 extracted
Start decoding Index 13098
Index 13098 extracted
Start decoding Index 13099
Index 13099 extracted
Start decoding Index 13101
Index 13101 extracted
Start decoding Index 13102
Index 13102 extracted
Start decoding Index 13104
Index 13104 extracted
Start decoding Index 13105
Index 13105 extracted
Start decoding Index 13107
Index 13107 extracted
Start decoding Index 13108
Index 13108 extracted
Start decoding Index 13110
Index 13110 extracte

In [ ]:
data_et = convertToNumeric(data_et, ['x', 'y', 't', 't_task'])

In [ ]:
data_et.groupby(['run_id', 'trial_index'])['x', 'y'].count()

# data_et_fixCal

In [ ]:
temp = data_et.merge(data_trial.loc[:, ['run_id', 'trial_index', 'trial_type']], 
                    on=['run_id', 'trial_index'],
                    how='left')

data_et_fixCal = temp.loc[temp['trial_type'].isin(['eyetracking-calibration', 'eyetracking-fix-object']), :] \
    .merge(data_trial.loc[:, ['run_id', 'trial_index', 'x_pos', 'y_pos']], 
          on=['run_id', 'trial_index'], 
          how='left') \
    .reset_index(drop=True)

# data_et_choice

In [ ]:
temp = data_et.merge(data_trial_choice.loc[:, 
                                           [
                                               'run_id', 'trial_index', 'trial_type',
                                               'amountLeft', 'LL_top', 'withinTaskIndex'
                                           ]
                                          ], 
                    on=['run_id', 'trial_index'],
                    how='left')

data_et_choice = temp.loc[temp['trial_type']=='eyetracking-choice', 
                          [
                              'run_id', 'trial_index', 
                              'x', 'y', 't', 't_task',
                              'amountLeft', 'LL_top', 'withinTaskIndex'
                          ]
                         ] \
    .reset_index(drop=True)

## Add variables

### Look direction

In [ ]:
def lookDirections(data):
    data["look_left"] = (data["x"] < 0.5).astype(int)
    data["look_top"] = (data["y"] < 0.5).astype(int)
    return data

data_et_choice = lookDirections(data_et_choice)

### AOIs

In [ ]:
def addAOI(data): 
    aoiCenters = pd.DataFrame(
        [
            [(0.05+0.9*0.2), 0.25],
            [(0.05+0.9*0.8), 0.25],
            [(0.05+0.9*0.2), 0.75],
            [(0.05+0.9*0.8), 0.75]
        ], 
        columns = ['x', 'y'],
        index = ['TL', 'TR', 'BL', 'BR']
    )
    
    data['aoi'] = 0 
    for aoi in aoiCenters.index:
        data.loc[
            (
                (data['x'] > (aoiCenters.loc[aoi, 'x'] - 0.175)) & \
                (data['x'] < (aoiCenters.loc[aoi, 'x'] + 0.175)) & \
                (data['y'] > (aoiCenters.loc[aoi, 'y'] - 0.175)) & \
                (data['y'] < (aoiCenters.loc[aoi, 'y'] + 0.175))
             ), 'aoi'] = aoi
    return data 

data_et_choice = addAOI(data_et_choice)
data_et_choice['aoi'].unique()

In [ ]:
def createAOIColumns(data):
    data['aoi_aLL'] = 0
    data['aoi_tLL'] = 0 
    data['aoi_aSS'] = 0 
    data['aoi_tSS'] = 0
    
    # If amounts are on the left side
    # If the gaze point is in the top option
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='TL')), 
             'aoi_aLL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='TR')), 
             'aoi_tLL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='BL')), 
             'aoi_aSS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='BR')), 
             'aoi_tSS'] = 1
    
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='TL')), 
             'aoi_aSS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='TR')), 
             'aoi_tSS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='BL')), 
             'aoi_aLL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='BR')), 
             'aoi_tLL'] = 1
    
    # If amounts are on the right side
    # If the gaze point is in the top option
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='TL')), 
             'aoi_tLL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='TR')), 
             'aoi_aLL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='BL')), 
             'aoi_tSS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='BR')),
             'aoi_aSS'] = 1

    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='TL')), 
             'aoi_tSS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='TR')), 
             'aoi_aSS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='BL')), 
             'aoi_tLL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='BR')), 
             'aoi_aLL'] = 1
    return data

data_et_choice = createAOIColumns(data_et_choice)

### Transitions

In [ ]:
def add_transition_type(data):
    data = data.loc[(pd.notna(data['aoi'])), :]
    data['newAOIIndex'] = 0
    data.loc[(data['aoi_aLL']==1), 'newAOIIndex'] = 1
    data.loc[(data['aoi_tLL']==1), 'newAOIIndex'] = 2
    data.loc[(data['aoi_aSS']==1), 'newAOIIndex'] = 4
    data.loc[(data['aoi_tSS']==1), 'newAOIIndex'] = 8
    data.sort_values(by=['run_id', 'withinTaskIndex'])
    # Add a 0 due to the way np.diff works
    data['transition_type'] = np.append(np.diff(data['newAOIIndex']), [0])
    data['transition_type'] = abs(data['transition_type']) 
    return(data)

def cleanTransitions(data):
    indices = []
    for subject in data['run_id'].unique():
        df_subj = data.loc[data['run_id']==subject, :]
        for trial in df_subj['withinTaskIndex'].unique():
            df_thisTrial = data.loc[(
                                    (data['run_id'] == subject) &
                                    (data['withinTaskIndex'] == trial)
                                ), 'transition_type']
            indices.append(df_thisTrial.index.max())
    # last gaze point of each trial
    data.loc[indices, 'transition_type'] = 0 
    return(data)

data_et_choice = add_transition_type(data_et_choice)

data_et_choice = cleanTransitions(data_et_choice)
# View(data_et_choice.tail(20))

data_et_choice

# data_subject

In [ ]:
data_subject = data_trial.loc[
    : ,
    [
        'chinFirst', 'choiceTask_amountLeftFirst', 'chosenAmount',
        'chosenDelay', 'run_id', 'prolificID'
    ]
] \
    .merge(data_raw.loc[:, ['run_id',
                            'age', 'browliner', 'browser', 'browser_version', 'degree', 'device',
                            'ethnic', 'eyeliner', 'eyeshadow', 'gender', 'glasses', 'keptHead',
                            'masquara', 'optionalNote', 'platform', 'platform_version', 'sight',
                            'triedChin', 'user_agent', 'vertPosition', 'webcam_fps',
                            'webcam_height', 'webcam_label', 'webcam_width']],
           on='run_id', 
           how='left') \
    .drop_duplicates()
data_subject.columns

## Add variables

### Prolific

In [ ]:
data_prolific = pd.read_csv(r'C:/Users/User/Google Drive/Masterarbeit/Acquise/prolific_documents/prolific_export.csv') \
    .rename(columns=
            {
                'participant_id': 'prolificID',
                'age': 'age_2'
            }
           )

data_subject = data_subject.merge(
    data_prolific, 
    on='prolificID', 
    how='left'
)

### Completion date

In [ ]:
if 'date' in data_subject.columns:
    print('date already added!')
else:
    output = []

    for subject in data_raw['run_id'].unique():
        thisSubject = data_raw.loc[data_raw['run_id']==subject] \
            .reset_index(drop=True)
        date_time_str=thisSubject.loc[0, 'recorded_at']
        date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')

        output.append([thisSubject.loc[0, 'run_id'], date_time_obj.date()])
        
    output = pd.DataFrame(output,
             columns=['run_id', 'date']
            )   
    data_subject = data_subject.merge(output, on='run_id', how='left')

# Exchange variables between the datasets

## data_trial_fixCal & data_trial_choice

### Count

In [ ]:
def merge_count_by_index(data, varName):
    if varName + '_count' in data.columns:
        print(varName + '_count already added!')
    else: 
        grouped = pd.DataFrame(
                data.groupby(["run_id", "trial_index"])[varName].count()
            ) \
            .reset_index() \
            .rename(columns={varName: varName + '_count'})
        data = data.merge(grouped, 
                          on=["run_id", "trial_index"], 
                          how='left')
    return(data)

temp = merge_count_by_index(data_et_fixCal, 'x')
data_trial_fixCal = data_trial_fixCal.merge(
    temp.loc[:, ['run_id', 'trial_index', 'x_count']].drop_duplicates(),
    on=['run_id', 'trial_index'],
    how='left'
)

temp = merge_count_by_index(data_et_choice, 'x')
data_trial_choice = data_trial_choice.merge(
    temp.loc[:, ['run_id', 'trial_index', 'x_count']].drop_duplicates(),
    on=['run_id', 'trial_index'],
    how='left'
)

### FPS

In [ ]:
data_trial_fixCal['fps'] = 1000 * data_trial_fixCal['x_count'] / data_trial_fixCal['trial_duration_exact'] 
data_trial_choice['fps'] = 1000 * data_trial_choice['x_count'] / data_trial_choice['trial_duration_exact']
data_trial_fixCal['fps'].describe()

## data_trial_choice

### Eye-Tracking indices

In [ ]:
def addOptionIndex(data):
    if "optionIndex" in data.columns:
        print("Option Index already added!")
        data_output = data
    else:
        grouped = data_et_choice.groupby(['run_id', 'trial_index']) \
            ['aoi', 'aoi_aSS', 'aoi_aLL', 'aoi_tSS', 'aoi_tLL'].count() \
            .reset_index() \
            .rename(columns={"aoi": "count"})
        grouped['gazePoints_immediate'] = (grouped['aoi_aSS'] + grouped['aoi_tSS'])
        grouped['gazePoints_delay'] = (grouped['aoi_aLL'] + grouped['aoi_tLL'])
        grouped['optionIndex'] = (grouped['gazePoints_immediate'] - grouped['gazePoints_delay']) / \
                                 (grouped['gazePoints_immediate'] + grouped['gazePoints_delay'])
        data_output = data.merge(grouped[['run_id', 'trial_index', 'optionIndex']], 
                                          on=['run_id', 'trial_index'])
    return(data_output)

data_trial_choice = addOptionIndex(data_trial_choice)

In [ ]:
def addAttributeIndex(data):
    if "attributeIndex" in data.columns:
        print('Attribute Index already added!')
        data_output = data
    else:
        grouped = data_et_choice.groupby(['run_id', 'trial_index']) \
            ['aoi', 'aoi_aSS', 'aoi_aLL', 'aoi_tSS', 'aoi_tLL'].count() \
            .reset_index() \
            .rename(columns={"aoi": "count"})
        grouped['gazePoints_amount'] = (grouped['aoi_aLL'] + grouped['aoi_aSS'])
        grouped['gazePoints_time'] = (grouped['aoi_tLL'] + grouped['aoi_tSS'])
        grouped['attributeIndex'] = (grouped['gazePoints_amount'] - grouped['gazePoints_time']) / \
                                 (grouped['gazePoints_amount'] + grouped['gazePoints_time'])

        data_output = data.merge(grouped[['run_id', 'trial_index', 'attributeIndex']], 
                                          on=['run_id', 'trial_index'])
    return(data_output)

data_trial_choice = addAttributeIndex(data_trial_choice)

transition_count = pd.pivot_table(data_et_choice.loc[:, ['run_id', 'trial_index', 'transition_type']], 
                    index = ['run_id', 'trial_index'],
                    columns = ['transition_type'], 
                    aggfunc = len,
                    fill_value = 0
                    ).reset_index()

In [ ]:
def addTransition_type(data_trial, data_et):
    if len(data_et.loc[:, 'transition_type'].unique())<2:
        transition_count = pd.DataFrame(
            columns = [
                'run_id', 'trial_index', 
                "trans_type_0",
                "trans_type_aLLtLL",
                "trans_type_tLLaSS",
                "trans_type_aLLaSS",
                "trans_type_aSStSS",
                "trans_type_tLLtSS",
                "trans_type_aLLtSS"
            ]
        )
    else:
        transition_count = pd.pivot_table(data_et.loc[:, ['run_id', 'trial_index', 'transition_type']], 
                            index = ['run_id', 'trial_index'],
                            columns = ['transition_type'], 
                            aggfunc = len,
                            fill_value = 0
                            ).reset_index()
        transition_count.columns = [
            'run_id', 'trial_index', 
            "trans_type_0",
            "trans_type_aLLtLL",
            "trans_type_tLLaSS",
            "trans_type_aLLaSS",
            "trans_type_aSStSS",
            "trans_type_tLLtSS",
            "trans_type_aLLtSS"
        ]

    if "trans_type_0" in data_trial:
        print("Transitions already added!")
        data_trial = data_trial
    else:
        data_trial = data_trial.merge(transition_count, on=['run_id', 'trial_index']) 
    return(data_trial)

data_trial_choice = addTransition_type(data_trial_choice, data_et_choice)

In [ ]:
def addPayneIndex(data):
    if "payneIndex" in data.columns:
        print("PayneIndex already added!")
    else: 
        # Option-wise: amount_LL-delay_LL = 1; amount_SS - delay_SS = 4
        # Attribute-wise: amount_LL-amount_SS = 3; delay_LL - delay_SS = 6
        # Cross: amount_LL-delay_SS = 7; delay_LL - amount_SS = 2
        optionWise_transition = data.loc[:, 'trans_type_aLLtLL'] + data.loc[:, 'trans_type_aSStSS']
        attributeWise_transition = data.loc[:, 'trans_type_aLLaSS'] + data.loc[:, 'trans_type_tLLtSS']  
        data['payneIndex'] = (optionWise_transition - attributeWise_transition) / \
            (optionWise_transition + attributeWise_transition)      
    return(data)

data_trial_choice = addPayneIndex(data_trial_choice)
data_trial_choice.columns

## data_subject

### fps

In [ ]:
temp = merge_count_by_index(data_et, 'x')
temp = data_trial.loc[:, ['run_id', 'trial_index', 'trial_duration_exact']] \
    .merge(temp.loc[:, ['run_id', 'trial_index', 'x_count']],
           on=['run_id', 'trial_index'], 
           how='left'
)
temp['fps'] = 1000 * temp['x_count'] / temp['trial_duration_exact']
temp['fps'].describe()

for column in ['trial_duration_exact', 'fps']:
    data_subject = merge_group_means_by_subject(data_subject, temp, column)    

### choseLL

In [ ]:
for column in ['choseLL']:
    data_subject = merge_group_means_by_subject(data_subject, data_trial_choice, column)    

# Export data

In [ ]:
if not os.path.exists('./data_jupyter'):
    os.mkdir('./data_jupyter')

data_et.to_csv("data_jupyter/data_et.csv", index=False, header=True)
data_et_fixCal.to_csv("data_jupyter/data_et_fixCal.csv", index=False, header=True)
data_et_choice.to_csv("data_jupyter/data_et_choice.csv", index=False, header=True)

data_trial.to_csv("data_jupyter/data_trial.csv", index=False, header=True)
data_trial_fixCal.to_csv("data_jupyter/data_trial_fixCal.csv", index=False, header=True)
data_trial_choice.to_csv("data_jupyter/data_trial_choice.csv", index=False, header=True)

data_subject.to_csv("data_jupyter/data_subject.csv", index=False, header=True)

MatLab input

In [ ]:
if not os.path.exists('./amasino_dataPrep/data_source'):
    os.mkdir('./amasino_dataPrep/data_source')

data_et_choice['fixationCounter'] = 1
data_et_choice.loc[:, 
                       [
                           'run_id', 
                           'withinTaskIndex', 
                           'x', 
                           'y', 
                           't_task'
                       ]
                  ] \
   .to_csv("amasino_dataPrep/data_source/schneegansEtAl_ET.csv", index=False, header=False)

In [ ]:
data_trial_choice.loc[:, 
                       [
                           'run_id', 
                           'withinTaskIndex', 
                           'optionIndex', 
                           'attributeIndex', 
                           'payneIndex'
                       ]
                  ] \
    .fillna(0) \
    .to_csv("amasino_dataPrep/intermediateCSVs/ET_indices.csv", index=False, header=False)

In [ ]:
data_trial_choice.loc[:, 
                          [
                              'run_id', 
                              'aSS', 
                              'aLL', 
                              'tSS', 
                              'tLL', 
                              'choseLL', 
                              'trial_duration_exact', 
                              'LL_top',
                              'choseTop'
                          ]
                     ] \
    .to_csv("amasino_dataPrep/data_source/schneegansEtAl_behavior.csv", index=False, header=False)

In [ ]:
data_subject.loc[:, ['run_id', 'choseLL']] \
    .to_csv("amasino_dataPrep/intermediateCSVs/percLeft.csv", index=False, header=False)

# Feedback

In [ ]:
print('Success! Script ran through')